# ACETONE tutorial #2

**Using other versions of a layer**

Efficiency is a key aspect in the embedded sector, with each code being specifically adpated to a target. As such, we need to be able to choose the implementation of each layer.

In this notebook, we'll explain how to use specific versions of a layer in ACETONE.

* When running this notebook on Colab, we need to install ACETONE 
* If you run this notebook locally, run it in the environment in which you installed ACETONE

The Python code on which the code for manipulating data from the MNIST database was originally written by Mélanie Ducoffe for the ANITI Tech'Session about the librairy DECOMON on 4/04/2025 ([replay](https://www.youtube.com/watch?v=1Jhj3xbsF1k&t=8s)). The code can be found in the [tutorial 3](https://airbus.github.io/decomon/main/tutorials.html) about local robustness to adversarial attacks for classification tasks.

In [ ]:
# TODO Installs on collab

In [ ]:
# Cleaning the working environment
from pathlib import Path
from os import remove, listdir

def clean_working_directory(directories):
    for directory in directories:
        if directory.exists():
            for file in listdir(directory):
                if not (directory / file).is_dir():
                    remove(directory / file)

# Path to the example files
PATH_DIR = Path("../tests/models/lenet5/lenet5_trained")

# Path to generated directories
loops_output_path = Path("demo_lenet_6loops")
indirect_gemm_output_path = Path("demo_lenet_indirect_gemm")
std_gemm_output_path = Path("demo_lenet_std_gemm")
demo_output_path = Path("demo_lenet_optimized")

files_directories = [loops_output_path, indirect_gemm_output_path, std_gemm_output_path, demo_output_path]

clean_working_directory(files_directories)

## Imports

In this notebook, we'll use as an example a simple Lenet5 model exported to Keras' format h5. The first four activation functions are hyperbolic tangents, while the last one is a softmax.

![lenet5](./data/lenet5_trained.png)

In [ ]:
# External imports
import numpy as np
import pystache

import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,8)

# ACETONE's imports
from acetone_nnet import CodeGenerator, cli_compare, list_all_implementations, conv2d_factory
from acetone_nnet.generator import Conv2D

## A few useful functions

We define two functions, one to write a set of inputs into an text file and the other to extract the C code's outputs.

In [ ]:
def write_input(inputs, nb_tests, file_name):
    """Writes a list of inputs in a text file.

    Args:
        inputs (list[np.ndarray]): The list of inputs to write
        nb_tests (int): The number of inputs
        file_name (str | Path): The path to the file in which the inputs will be written

    Raises:
        ValueError: Can't write more tests (nb_tests) than the number of given inputs
    """
    if nb_tests > len(inputs):
        raise ValueError(f"Can't have more tests to write ({nb_tests}) than values ({len(inputs)})")
    with open(file_name, "w+") as fi:
        for i in range(nb_tests):
            x = inputs[i].flatten()
            out_string = " ".join(
                [f'{float(n).hex().replace("0000000p", "p")}' for n in x],
            )
            print(f"{out_string}", file=fi, flush=True)


def extract_outputs_c(
        path_to_output: str | Path,
        nb_tests: int,
) -> np.ndarray:
    """Get the outputs values from the txt file."""
    output = []
    with Path.open(Path(path_to_output)) as f:
        for i, line in enumerate(f):

            line = line[:-2].split(" ")
            line = list(map(float.fromhex, line))
            line = np.array(line)
            
            output.append(line)
            
            if i >= nb_tests:
                break
    f.close()
    return np.array(output)

## Loading images

We first load MNIST data from keras dataset.

In [ ]:
img_rows, img_cols = 28, 28
(x_train, y_train_), (x_test, y_test_) = mnist.load_data()
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255.0
x_test /= 255.0
y_train = keras.utils.to_categorical(y_train_)
y_test = keras.utils.to_categorical(y_test_)

x_train = np.array([x.reshape((28,28)) for x in x_train])
x_test = np.array([x.reshape((28,28)) for x in x_test])

## Generating a test input

We select a few random images to use as support for the later parts of the notebook.

In [ ]:
nb_col = 5
nb_row = 2
nb_samples = nb_col*nb_row
indexes = np.random.permutation(len(x_test))[:nb_samples]

inputs = x_test[indexes]

fig,axs = plt.subplots(nb_row,nb_col)

for i in range(nb_row):
    for j in range(nb_col):
        ax = axs[i,j]
        ax.imshow(inputs[i*nb_col+j], cmap="gray")

## Defining common parameters and training the model

We then prepare the CodeGenerator's initialization parameters and train the model.

In [ ]:
# Path to the model
model_path = PATH_DIR / "lenet5_trained.h5"
model = keras.models.load_model(model_path)
model.compile("adam", "categorical_crossentropy", metrics=["acc"])

In [ ]:
model.fit(x_train, y_train, batch_size=32, shuffle=True, validation_split=0.2, epochs=3)
model.evaluate(x_test, y_test, batch_size=32)

The model has an accuracy of `0.98`, which means that out of 100 images, 98 will be well labeled. we will now generate the C code while preserving its semantics.

With our test inputs, we have:

In [ ]:
# Inference
keras_outputs = model.predict(inputs)
# The index of the highest scores represents the label
keras_labels = keras_outputs.argmax(axis=-1)

print(f"The labels are: {keras_labels}")

In [ ]:
# Path to the input file
dataset = Path("./lenet_inputs.txt")
write_input(inputs, nb_samples, dataset)
# Function name
function_name = "demo_lenet"
# Number of tests
nb_tests = nb_samples

## Using ACETONE's native implementations

The framework laready provides, for some layers, several versions from which to choose before generating our code.

In [ ]:
implemented = list_all_implementations()
for layer_name in implemented:
    print(layer_name,":")
    for implementation in implemented[layer_name]:
        print("   ", implementation)
    print("\n")

## Create an ACETONE CodeGenerator from the model

Let's focus on the convolution layer.

The 3 main algorithms provided by ACETONE for the convolution are: *6loops*, *std_gemm* and *indirect_gemm*.

The first algorithm implements the naive version of a convolution: a triple loop iters all the indices of the ouput, and for each of this indices another triple loop compute the value using the input tensor and the kernel, as illustrated below (for a 2D example).

![Conv_6loops](./data/conv_6loops.gif "segment")

(Figure excerpt from [https://www.geeksforgeeks.org/apply-a-2d-convolution-operation-in-pytorch/](https://www.geeksforgeeks.org/apply-a-2d-convolution-operation-in-pytorch/))

We can change the implementation of a specific type of layer by using the class **CodeGenerator**'s argument `versions`.
This argument takes a dictionary containing a reference to the layer (usually the name) as key and the version's name as value.

In [ ]:
# Create an ACETONE CodeGenerator from the model
loops_generator = CodeGenerator(
    file=model,
    function_name=function_name,
    external_input=True,
    versions={"Conv2D":"6loops"},
    nb_tests=nb_tests,
    verbose=False,
)

In [ ]:
loops_generator.generate_c_files(loops_output_path)

The second algorithm, *std_gemm*, uses the standard gemm (as the name suggests) to compute the convolution. In the generated C code, the convolution is divided in two main parts: the creation of the matrix of patches from the input then the multiplication of the resulting matrix with the kernel matrix.

![Conv_std_gemm](./data/std_gemm.png)

(Figure excerpt from [*Extending a predictable machine learning framework with efficient gemm-based convolution routines*](https://theses.hal.science/ONERA-MIP/hal-04627347v1) written by Iryna De Albuquerque Silva, Thomas Carle, Adrien Gauffriau and Claire Pagetti)

In [ ]:
# Create an ACETONE CodeGenerator from the model
std_gemm_generator = CodeGenerator(
    file=model,
    function_name=function_name,
    external_input=True,
    versions={"Conv2D":"std_gemm_nn"},
    nb_tests=nb_tests,
    verbose=False,
)

In [ ]:
std_gemm_generator.generate_c_files(std_gemm_output_path)

The third algorithm is another implementation of the gemm algorithm. Instead of computing the patches matrix during the execution of the C code, the framework computes before the generation and adds to the code a matrix of pointers, each referring to the corresponding element in the input tensor. The `im2col` operation is then included in the pointer matrix instead of being computed during execution.

![Conv_indirect_gemm](./data/indirect_gemm.png)

(Figure excerpt from [*Extending a predictable machine learning framework with efficient gemm-based convolution routines*](https://theses.hal.science/ONERA-MIP/hal-04627347v1) written by Iryna De Albuquerque Silva, Thomas Carle, Adrien Gauffriau and Claire Pagetti)

In [ ]:
# Create an ACETONE CodeGenerator from the model
indirect_gemm_generator = CodeGenerator(
    file=model,
    function_name=function_name,
    external_input=True,
    versions={"Conv2D":"indirect_gemm_nn"},
    nb_tests=nb_tests,
    verbose=False,
    )

In [ ]:
indirect_gemm_generator.generate_c_files(indirect_gemm_output_path)

We can now compile and run those generated code to ensure that the semantic was indeed preserved.

In [ ]:
# Compiling the code
! make -C demo_lenet_6loops all

# Running the executable
! ./demo_lenet_6loops/demo_lenet ./demo_lenet_6loops/output_c.txt ./lenet_inputs.txt

In [ ]:
# Compiling the code
! make -C demo_lenet_std_gemm all

# Running the executable
! ./demo_lenet_std_gemm/demo_lenet ./demo_lenet_std_gemm/output_c.txt ./lenet_inputs.txt

In [ ]:
# Compiling the code
! make -C demo_lenet_indirect_gemm all

# Running the executable
! ./demo_lenet_indirect_gemm/demo_lenet ./demo_lenet_indirect_gemm/output_c.txt ./lenet_inputs.txt

In [ ]:
cli_compare(reference_file=(indirect_gemm_output_path / "output_c.txt"), c_file=(std_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)

In [ ]:
loops_output = extract_outputs_c(path_to_output=(loops_output_path / "output_c.txt"), nb_tests=nb_tests)
std_gemm_output = extract_outputs_c(path_to_output=(std_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)
indirect_gemm_output = extract_outputs_c(path_to_output=(indirect_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)

loops_labels = loops_output.argmax(axis = -1)
std_gemm_labels = std_gemm_output.argmax(axis = -1)
indirect_gemm_labels = indirect_gemm_output.argmax(axis = -1)


print("Labels given by the inference:")
for i in range(nb_row):
    for j in range(nb_col):
        ax = axs[i,j]
        ax.imshow(inputs[i*nb_col+j], cmap="gray")
        print(f"sample at position ({i},{j}) : ")
        print(f"    keras :         {keras_labels[i*nb_col+j]}")
        print(f"    6loops :        {loops_labels[i*nb_col + j]}")
        print(f"    std_gemm :      {std_gemm_labels[i*nb_col + j]}")
        print(f"    indirect_gemm : {indirect_gemm_labels[i*nb_col + j]}")

The generated code gives indeed the same label for all the implementations.

# Playing a bit more

Let's now try with several set of inputs from the MNIST dataset.

In [ ]:
# Selecting new inputs
indexes = np.random.permutation(len(x_test))[:nb_samples]
inputs = x_test[indexes]
write_input(inputs, nb_samples, dataset)

# Keras inference and labels
keras_outputs = model.predict(inputs)
keras_labels = keras_outputs.argmax(axis=-1)

# ACETONE's inference with the 3 versions
! ./demo_lenet_6loops/demo_lenet ./demo_lenet_6loops/output_c.txt ./lenet_inputs.txt
! ./demo_lenet_std_gemm/demo_lenet ./demo_lenet_std_gemm/output_c.txt ./lenet_inputs.txt
! ./demo_lenet_indirect_gemm/demo_lenet ./demo_lenet_indirect_gemm/output_c.txt ./lenet_inputs.txt

# Extracting the scores
loops_output = extract_outputs_c(path_to_output=(loops_output_path / "output_c.txt"), nb_tests=nb_tests)
std_gemm_output = extract_outputs_c(path_to_output=(std_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)
indirect_gemm_output = extract_outputs_c(path_to_output=(indirect_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)

# Getting the labels
loops_labels = loops_output.argmax(axis = -1)
std_gemm_labels = std_gemm_output.argmax(axis = -1)
indirect_gemm_labels = indirect_gemm_output.argmax(axis = -1)

# Comparing the results
max_rel_error = 0.0
max_abs_error = 0.0
for i in range(nb_samples):
    for k in range(10):
        diff = abs(keras_outputs[i,k] - loops_output[i,k])
        norm = abs(keras_outputs[i,k]) + abs(loops_output[i,k])
        max_abs_error = max(max_abs_error , diff)
        if norm != 0:
            max_rel_error = max(max_rel_error, diff/(norm/2))

print("Difference between loops implementation and standard gemm implementation:")
cli_compare(reference_file=(loops_output_path / "output_c.txt"), c_file=(std_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)
print("Difference between indirect gemm implementation and standard gemm implementation:")
cli_compare(reference_file=(indirect_gemm_output_path / "output_c.txt"), c_file=(std_gemm_output_path / "output_c.txt"), nb_tests=nb_tests)

print("Comparing Keras and ACETONE's results :")
print("Maximal absolute error between them : ",max_abs_error)
print("Maximal relative error between them : ",max_rel_error)

fig,axs = plt.subplots(nb_row,nb_col)
fig.suptitle("Label given by : Keras-6loops-std_gemm-indirect_gemm")
for i in range(nb_row):
    for j in range(nb_col):
        ax = axs[i,j]
        ax.imshow(inputs[i*nb_col+j], cmap="gray")
        ax.set_title(f"{keras_labels[i*nb_col+j]}-{loops_labels[i*nb_col+j]}-{std_gemm_labels[i*nb_col+j]}-{indirect_gemm_labels[i*nb_col+j]}")



All four labels are the same, the semantic is preserved !!